# Auto scout project Balint Part 2: dealing with missing values

In [50]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [51]:
# load in the cleaned data 
df = pd.read_csv('data/clean_auto_scout.csv')

In [52]:
df.head()

,make_model,price,previous_owners_cleaned,co2emission_cleaned_gco2/km,emissionlabel_cleaned,emissionclass_cleaned,body_cleaned,bodycolor_cleaned,km_cleaned,vat_cleaned,...,led_headlights,lane_departure_warning_system,passenger-side_airbag,power_steering,rear_airbag,side_airbag,tire_pressure_monitoring_system,traction_control,traffic_sign_recognition,xenon_headlights
0,Audi A1,15770,2.0,992.0,NaN,6,Sedans,Black,56013.0,VAT deductible,...,0,0,1,1,0,1,1,1,0,1
1,Audi A1,14500,NaN,1292.0,Green,6,Sedans,Red,80000.0,Price negotiable,...,0,0,1,1,0,1,1,1,0,1
2,Audi A1,14640,1.0,992.0,Green,6,Sedans,Black,83450.0,VAT deductible,...,0,0,1,1,0,1,1,1,0,0
3,Audi A1,14500,1.0,992.0,NaN,6,Sedans,Brown,73000.0,NaN,...,0,0,1,1,0,1,1,0,0,0
4,Audi A1,16790,1.0,1092.0,NaN,6,Sedans,Black,16200.0,NaN,...,0,0,1,1,0,1,1,1,0,1


## General strategy: find colinear variables and groupby these, calculate some average on the variable to fill 

In [53]:
# Search for potential groupings of variables
(dict(df.dtypes))

{'make_model': dtype('O'),
 'price': dtype('int64'),
 'previous_owners_cleaned': dtype('float64'),
 'co2emission_cleaned_gco2/km': dtype('float64'),
 'emissionlabel_cleaned': dtype('O'),
 'emissionclass_cleaned': dtype('O'),
 'body_cleaned': dtype('O'),
 'bodycolor_cleaned': dtype('O'),
 'km_cleaned': dtype('float64'),
 'vat_cleaned': dtype('O'),
 'registration_cleaned': dtype('O'),
 'hp_kw_cleaned': dtype('float64'),
 'nextinspection_cleaned': dtype('O'),
 'type_cleaned': dtype('O'),
 'fuel_cleaned': dtype('O'),
 'particulate_filter': dtype('int64'),
 'multiple_fuel': dtype('int64'),
 'offernumber_cleaned': dtype('O'),
 'warranty_months': dtype('float64'),
 'painttype_cleaned': dtype('O'),
 'interior_material': dtype('O'),
 'interior_colour': dtype('O'),
 'nrofdoors_cleaned': dtype('float64'),
 'nrofseats_cleaned': dtype('float64'),
 'modelcode_cleaned': dtype('O'),
 'gearingtype_cleaned': dtype('O'),
 'gears_cleaned': dtype('float64'),
 'cylinders_cleaned': dtype('float64'),
 'drivec

In [54]:
# num_cols = df[]

In [55]:
# Determine columns with missing values
cols_with_mv = df.columns[df.isna().sum() > 0]
print(sorted(cols_with_mv))

['body_cleaned', 'bodycolor_cleaned', 'co2emission_cleaned_gco2/km', 'consumption_city_l_per_100km', 'consumption_comb_l_per_100km', 'consumption_country_l_per_100km', 'countryversion_cleaned', 'cylinders_cleaned', 'displacement_cleaned_cc', 'drivechain_cleaned', 'emissionclass_cleaned', 'emissionlabel_cleaned', 'gears_cleaned', 'hp_kw_cleaned', 'interior_colour', 'interior_material', 'km_cleaned', 'modelcode_cleaned', 'nextinspection_cleaned', 'nrofdoors_cleaned', 'nrofseats_cleaned', 'offernumber_cleaned', 'painttype_cleaned', 'previous_owners_cleaned', 'registration_cleaned', 'type_cleaned', 'vat_cleaned', 'warranty_months', 'weight_cleaned_kg']


## registration_cleaned

In [56]:
df['registration_cleaned'].isna().sum()

np.int64(1597)

Generate a car 'age' column from the registration data year

In [57]:
# Extract year info to determine cars age
df['registration_cleaned'] = pd.to_datetime(df['registration_cleaned']) # convert to datetime type,looks like i did this already in part1, maybe converting to csv loses this
df['year'] = df['registration_cleaned'].dt.year
df['year'].value_counts()
df.drop('registration_cleaned', axis = 1, inplace = True)

In [58]:
# Generate 'age' column where 2019 is new (age = 0)
# Define the mapping of year to age
year_to_age = {
    2019: 0,
    2018: 1,
    2017: 2,
    2016: 3
}
# Use map() to apply the mapping
df['age'] = df['year'].map(year_to_age)
df['age'].value_counts(dropna = False)

age
1.0    4522
3.0    3674
2.0    3273
0.0    2853
NaN    1597
Name: count, dtype: int64

##

## km_cleaned column

In [64]:
median_group_km = df.groupby(['make_model','body_cleaned','age'])['km_cleaned'].transform(lambda x: x.median())

df['km_cleaned_filled'] = df['km_cleaned'].fillna(median_group_km)
df['km_cleaned_filled'].value_counts(dropna = False)

c:\Users\BalintStewart\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


km_cleaned_filled
10.0       1076
NaN         841
1.0         367
5.0         174
50.0        167
           ... 
6250.0        1
22801.0       1
28976.0       1
27896.0       1
69435.0       1
Name: count, Length: 6692, dtype: int64

In [81]:
df[['km_cleaned_filled', 'km_cleaned']].sample(20)
df[df['km_cleaned'].isna()]['km_cleaned_filled'].value_counts(dropna = False)

km_cleaned_filled
NaN        841
36.0        41
10.0        31
50.0        19
13587.5     14
1500.0      12
178.0        8
15.0         7
500.0        7
7000.0       6
10564.0      5
2000.0       5
11.0         4
5.0          4
12000.0      3
775.5        3
3000.0       3
10780.0      3
19800.0      2
18000.0      1
7.0          1
25150.0      1
35.0         1
31000.0      1
28000.0      1
Name: count, dtype: int64